In [10]:
import networkx as nx
import math

# This function computes a lower bound on the length of Hamiltonian cycles starting with vertices in the list sub_cycle.
def lower_bound(g, sub_cycle):
    current_weight = sum([g[sub_cycle[i]][sub_cycle[i + 1]]['weight'] for i in range(len(sub_cycle) - 1)])

    unused_nodes = [v for v in g.nodes() if v not in sub_cycle]
    subgraph = g.subgraph(unused_nodes)
    mst_edges = list(nx.minimum_spanning_edges(subgraph))
    mst_weight = sum([subgraph.get_edge_data(e[0], e[1])['weight'] for e in mst_edges])

    if len(sub_cycle) == 0 or len(sub_cycle) == g.number_of_nodes():
        return mst_weight + current_weight
    else:
        f_vertex = sub_cycle[0]
        l_vertex = sub_cycle[-1]

        min_to_f_weight = min([g[v][f_vertex]['weight'] for v in g.nodes() if v not in sub_cycle])
        min_by_l_weight = min([g[l_vertex][v]['weight'] for v in g.nodes() if v not in sub_cycle])

        return current_weight + min_by_l_weight + mst_weight + min_to_f_weight

In [15]:
# The branch and bound procedure takes
def branch_and_bound(g, sub_cycle=None, current_min=float("inf")):
    # If the current path is empty, then we can safely assume that it starts with the vertex 0.
    if sub_cycle is None:
        sub_cycle = [0]

    # If we already have all vertices in the cycle, then we just compute the weight of this cycle and return it.
    if len(sub_cycle) == g.number_of_nodes():
        return sum([g[sub_cycle[i]][sub_cycle[i + 1]]['weight'] for i in range(len(sub_cycle) - 1)]) \
               + g[sub_cycle[-1]][sub_cycle[0]]['weight']

    # Now we look at all nodes which aren't yet used in sub_cycle.
    unused_nodes = list()
    for v in g.nodes():
        if v not in sub_cycle:
            unused_nodes.append((g[sub_cycle[-1]][v]['weight'], v))

    # We sort them by the distance from the "current node" -- the last node in sub_cycle.
    unused_nodes = sorted(unused_nodes)

    for (d, v) in unused_nodes:
        assert v not in sub_cycle
        extended_subcycle = list(sub_cycle)
        extended_subcycle.append(v)
        # For each unused vertex, we check if there is any chance to find a shorter cycle if we add it now.
        if lower_bound(g, extended_subcycle) < current_min:
            # If there is such a chance, we add the vertex to the current cycle, and proceed recursively.
            # If we found a short cycle, then we update the current_min value.
            weight = branch_and_bound(g, extended_subcycle, current_min)
            if weight < current_min:
                current_min = weight

    return current_min  # The procedure returns the shortest cycle length.

In [7]:
# This function computes the distance between two points.
def dist(x1, y1, x2, y2):
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

In [8]:
# This function receives a list of 2-tuples representing the points' coordinates,
# and returns the corresponding graph.
def get_graph(coordinates):
    g = nx.Graph()
    n = len(coordinates)
    for i in range(n):
        for j in range(i + 1):
            g.add_edge(i, j, weight=dist(coordinates[i][0], coordinates[i][1], coordinates[j][0], coordinates[j][1]))
    return g

In [17]:
coords = [(178, 212), (287, 131), (98, 156)]  # correct answer is 424.1 ...
G = get_graph(coords)
branch_and_bound(G)

424.1000397032701